In [4]:
import numpy as np
from sympy import *
import random
import time

### Вспомогательная функция

In [5]:
def roots_to_dict(roots, x):
    dict_roots = dict()
    for i in range(len(roots)):
        dict_roots[x[i]] = roots[i]

    return dict_roots

### Якобиан

In [6]:
def get_jacobi(system_equations: np.array):
    n = system_equations.shape[0]
    x = symbols(f'x:{n}')
    J = np.empty(shape=(n, n), dtype=core.add.Add)
    for i in range(n):
        for j in range(n):
            J[i, j] = system_equations[i].diff(x[j])

    return J

### Метод простых итераций

In [7]:
def get_q(fi_equation, approx, e=0.1):
    x_0 = approx[0]
    x_fi = approx[1]
    x = symbols('x:2')
    e = 0.1
    x_1 = random.uniform(x_0 - e, x_0 + e)
    x_2 = random.uniform(x_0 - e, x_0 + e)

    q = (abs(fi_equation.subs({x[0]: x_1, x[1]: x_fi}) - 
             fi_equation.subs({x[0]: x_2, x[1]: x_fi}))) / (abs(x_1 - x_2))
    return q

In [19]:
def iteration_solve(system_equations: np.array, approx, tol=0.00001):
    print('\nIteration method computing...')

    n = system_equations.shape[0]
    x = symbols(f'x:{n}')

    fi_equations = system_equations[1]

    curr_roots = list(approx)

    errors = np.zeros(shape=(n, ))
    error = tol * 10000

    try:
        q_1 = float(get_q(fi_equations[0], (approx[0], approx[1]), 0.01))
        q_2 = float(get_q(fi_equations[1], (approx[1], approx[0]), 0.01))
    except:
        print("Q is not real, cant solve")
        return None, None
    print(f'q_1 = {q_1:.4f}')
    print(f'q_2 = {q_2:.4f}')
    if q_1>=1 or q_2>=1:
        print("q is greater than 1")
        return None, None

    iteration = 0
    while error > tol:
        prev_roots = curr_roots.copy()
        roots_d = roots_to_dict(curr_roots, x)
        print(fi_equations)
        for i in range(n):
            try:
                curr_roots[i] = float(fi_equations[i].subs(roots_d))
            except TypeError:
                print("some complex numbers")

            errors[i] = abs(prev_roots[i] - curr_roots[i])
            roots_d = roots_to_dict(curr_roots, x)

        error = np.amax(errors)
        iteration += 1


    print('stopped iteration method\n')

    return curr_roots, iteration

### Метод Ньютона

In [9]:
import numpy as np
from sympy import symbols
from staff_matrix import get_jacobi, roots_to_dict


def newton_solve(system_equations: np.array, approx, tol=0.00001):
    n = system_equations.shape[0]
    x = symbols(f'x:{n}')

    J = get_jacobi(system_equations)

    error = tol * 10000
    iteration = 0
    roots = approx
    while error > tol:
        iteration += 1

        roots_d = roots_to_dict(roots, x)
        jacobi_values = np.zeros(shape=(n, n))
        for i in range(n):
            for j in range(n):
                jacobi_values[i, j] = J[i, j].subs(roots_d)

        jacobi_det = np.linalg.det(jacobi_values)
        print(f"Jacobi det = {jacobi_det:.4f}")
        if not jacobi_det:
            print("det equal 0. Can't solve system")
            exit(0)

        F = np.zeros(shape=(n, ))
        for i in range(0, n):
            F[i] = system_equations[i].subs(roots_d)

        delta_x = np.linalg.solve(jacobi_values, -1 * F)

        roots = delta_x + roots
        error = np.amax(abs(delta_x))

    return roots, iteration

In [10]:
def print_list(l):
    try:
        for el in l:
            print(f"{el:0.4f}",end=' ')
    except:
        print("EMPTY")
    return ''

In [11]:
def get_system_1_it():
    global y_1_1, y_1_2
    return np.array([
        [
            y_1_1,
            y_1_2
        ],
        [
            sqrt((x[0] * x[1] + 5*x[0] - 1) / 2),
            sqrt(x[0]+3*log(x[0], 10))
        ]
        ])

### Тестовый пример 1

In [12]:
x = symbols('x:2')

In [13]:
y_1_1 = 2*x[0]**2 - x[0]*x[1] - 5*x[0] + 1
y_1_2 = x[0] + 3*log(x[0], 10) - x[1]**2
approx = (3.5, 2.2)

In [14]:
def get_system_1_it():
    global y_1_1, y_1_2
    return np.array([
        [
            y_1_1,
            y_1_2
        ],
        [
            sqrt((x[0] * x[1] + 5*x[0] - 1) / 2),
            sqrt(x[0]+3*log(x[0], 10))
        ]
        ])

In [15]:
system = get_system_1_it()

In [20]:
print(f"\t\tНачальное приближение: {approx}")
start_time = time.time()
iteration_ans = iteration_solve(system, approx)
print(f"Время выполнения метода итераций {time.time() - start_time:0.5f} секунд")
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
start_time = time.time()
newton_ans = newton_solve(system[0], approx)
print(f"Время выполнения метода ньютона {time.time() - start_time:0.5f} секунд")
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения:")
print_list(newton_ans[0])
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (3.5, 2.2)

Iteration method computing...
q_1 = 0.5164
q_2 = 0.4500
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
[sqrt(x0*x1/2 + 5*x0/2 - 1/2) sqrt(x0 + 3*log(x0)/log(10))]
stopped iteration method

Время выполнения метода итераций 0.07591 

### Тестовый пример 2

In [14]:
y_2_1 = x[0]**2 + x[0]*x[1] - 10
y_2_2 = x[1] + 3*x[0]*x[1]**2 - 57
approx = (1.5, 3.5)


In [15]:
def get_system_2_it():
    global y_2_1, y_2_2
    return np.array([
        [
            y_2_1,
            y_2_2
        ],
        [
            sqrt(10 - x[0]*x[1]),
            sqrt((57-x[1]) / (3*x[0]))
        ]
    ])

In [16]:
system = get_system_2_it()
print(f"\t\tНачальное приближение: {approx}")
start_time = time.time()
iteration_ans = iteration_solve(system, approx)
print(f"Время выполнения метода итераций {time.time() - start_time:0.5f} секунд")
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
start_time = time.time()
newton_ans = newton_solve(system[0], approx)
print(f"Время выполнения метода ньютона {time.time() - start_time:0.5f} секунд")
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения:")
print_list(newton_ans[0])
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (1.5, 3.5)

Iteration method computing...
q_1 = 0.8022
q_2 = 0.3271
stopped iteration method

Время выполнения метода итераций 0.12488 секунд
		*** Метод простой итерации: ***
2.0000 3.0000 Корни уравнения: 
Количество итераций: 12
--------------------
--------------------
		Начальное приближение: (1.5, 3.5)
Jacobi det = 156.1250
Jacobi det = 197.7843
Jacobi det = 204.9696
Jacobi det = 204.9999
Время выполнения метода ньютона 0.03259 секунд
		*** Метод Ньютона: ***
Корни уравнения:
2.0000 3.0000 Количество итераций: 4


### Тестовый пример 3

In [17]:
y_3_1 = x[0]**2 - x[1] - 1
y_3_2 = x[0] - x[1]**2 + 1
approx = (1.9, 1.8)

In [18]:
def get_system_3_it():
    global y_3_1, y_3_2
    return np.array([
        [
            y_3_1,
            y_3_2
        ],
        [
            sqrt(1+x[1]),
            sqrt(1 + x[0])
        ]
    ])

In [19]:
system = get_system_3_it()
print(f"\t\tНачальное приближение: {approx}")
start_time = time.time()
iteration_ans = iteration_solve(system, approx)
print(f"Время выполнения метода итераций {time.time() - start_time:0.5f} секунд")
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
start_time = time.time()
newton_ans = newton_solve(system[0], approx)
print(f"Время выполнения метода ньютона {time.time() - start_time:0.5f} секунд")
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения:")
print_list(newton_ans[0])
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (1.9, 1.8)

Iteration method computing...
q_1 = 0.0000
q_2 = 0.2963
stopped iteration method

Время выполнения метода итераций 0.01091 секунд
		*** Метод простой итерации: ***
1.6180 1.6180 Корни уравнения: 
Количество итераций: 6
--------------------
--------------------
		Начальное приближение: (1.9, 1.8)
Jacobi det = -12.6800
Jacobi det = -9.7416
Jacobi det = -9.4747
Jacobi det = -9.4721
Время выполнения метода ньютона 0.02146 секунд
		*** Метод Ньютона: ***
Корни уравнения:
1.6180 1.6180 Количество итераций: 4


### Задание

In [20]:
m = 0.1
a = 0.7

approx = (0.6, 0.6)

y_1 = tan(x[0]*x[1] + m) - x[0]
y_2 = a*x[0]**2 + 2*x[1]**2 - 1

def get_system():
    return np.array([
        [
            y_1,
            y_2
        ],
        [
            tan(x[0]*x[1] + m),
            sqrt((1 - a*x[0]**2) / 2)
        ]
    ])

In [21]:
system = get_system()

print(f"\t\tНачальное приближение: {approx}")
start_time = time.time()
iteration_ans = iteration_solve(system, approx)
print(f"Время выполнения метода итераций {time.time() - start_time:0.5f} секунд")
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
start_time = time.time()
newton_ans = newton_solve(system[0], approx)
print(f"Время выполнения метода ньютона {time.time() - start_time:0.5f} секунд")
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения:")
print_list(newton_ans[0])
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (0.6, 0.6)

Iteration method computing...
q_1 = 0.7172
q_2 = 0.3685
stopped iteration method

Время выполнения метода итераций 0.27596 секунд
		*** Метод простой итерации: ***
0.3499 0.6761 Корни уравнения: 
Количество итераций: 25
--------------------
--------------------
		Начальное приближение: (0.6, 0.6)
Jacobi det = -1.2342
Jacobi det = -0.8602
Jacobi det = -0.8408
Jacobi det = -0.8444
Время выполнения метода ньютона 0.05438 секунд
		*** Метод Ньютона: ***
Корни уравнения:
0.3499 0.6761 Количество итераций: 4
